# Justin Meyer        CS 5310 Ch. 3 Lab - kNN

In this lab I will do some data preprocessing to control for multicollinearity and then normalize the data set before running the kNN algorithm

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Import the Features data sets

df1 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/BG_features.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/FZ_features.csv')
df3 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/MJ_features.csv')
df4 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/MR_features.csv')
df5 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/YH_features.csv')
Features_df = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [ ]:
# Import the Objects data sets

df1 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/BG_objects.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/FZ_objects.csv')
df3 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/MJ_objects.csv')
df4 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/MR_objects.csv')
df5 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-3/YH_objects.csv')
Objects_df = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [ ]:
# Create the correlation matrix and convert values to absolute values

cor_output = Features_df.corr(method='pearson', min_periods=1).abs()

In [ ]:
# Create come logic to quickly remove all values correlated that is above .80

mask = np.triu(np.ones_like(cor_output, dtype=bool))
tri_df = cor_output.mask(mask)

to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.80)]

reduced_df = Features_df.drop(to_drop, axis=1)

In [ ]:
# Normalize the features using the preprocessing MinMaxScaler from sklearn

from sklearn import preprocessing
x = reduced_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_df = pd.DataFrame(x_scaled)

In [ ]:
# Add back in the column headers

col_names = pd.DataFrame(reduced_df.columns)
normalized_df.columns = col_names

In [ ]:
# Fixes an issue with the indexing for the Objects dataframe

Objects_df.reset_index(drop=True, inplace=True)

In [ ]:
# Combine the dataframes

Full_df = Objects_df.join(normalized_df)
print(Full_df)

      # Object  (beta0,)  ...  (approx_entropy4,)  (Hurst4,)
0            3  0.157238  ...            0.048792   0.907102
1            2  0.151668  ...            0.129909   0.923736
2            5  0.121261  ...            0.475110   0.620028
3            1  0.151377  ...            0.191514   0.924104
4            1  0.177493  ...            0.062870   0.983125
...        ...       ...  ...                 ...        ...
1265         2  0.075409  ...            0.127790   0.891287
1266         1  0.087652  ...            0.185483   0.882339
1267         2  0.098463  ...            0.087741   0.716462
1268         1  0.704196  ...            0.109323   0.947004
1269         3  0.112723  ...            0.106556   0.749792

[1270 rows x 51 columns]


In [ ]:
# Split the data into features and labels

X = np.array(Full_df.drop(['# Object'],1))
y = np.array(Full_df['# Object'])

In [ ]:
# Split the data into training data set and testing data set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
# Train the kNN model and output the crosstab

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(X_train, y_train)
nuro_test_pred = neigh.predict(X_test)
pd.crosstab(y_test, nuro_test_pred)

col_0,1,2,3,4,5
row_0,,,,,
1,19,8,15,6,7
2,22,11,7,8,8
3,18,8,7,11,7
4,19,7,10,6,4
5,15,12,5,5,9


In [ ]:
# How accurate is the model?

accuracy = neigh.score(X_test, y_test)
print(accuracy)

0.2047244094488189


In [ ]:
from matplotlib import pyplot as plt
plt.plot()

# Conclusion

The model was only 20% accurate in predicting which object belonged to which set of features